<h3 align="left" style='color:blue'>Optimize tensorflow pipeline performance with prefetch and caching</h3>

In [54]:
import tensorflow as tf
import time

In [55]:
tf.__version__

'2.14.1'

<h3 style='color:purple'>Prefetch</h3>

In [159]:
class FileDataset(tf.data.Dataset):
    def read_file_in_batches(num_samples):
        # Opening the file
        print('      ------------------------------------------------------------------------------')
        print('      3. Opening the File => num_samples:', num_samples)
        time.sleep(0.03)

        for sample_idx in range(num_samples):
            # Reading data (line, record) from the file
            print('        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr')
            print('        4. Reading data => sample_idx:', sample_idx)
            time.sleep(0.015)
            #print()

            yield (sample_idx,)

    def __new__(cls, num_samples=3):
        print("***********************************************************************************")
        print("1. __new__ => cls:", cls, " num_samples:", num_samples)
        return tf.data.Dataset.from_generator(
            cls.read_file_in_batches,
            output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args=(num_samples,)
        )

In [160]:
def benchmark(dataset, num_epochs=2):  # training deplay
    for epoch_num in range(num_epochs):
        print('    ===============================================================================')
        print('    2. epoch_num:', epoch_num)
        for sample in dataset:
            # Performing a training step
            print('        ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt')
            print('        5. Performing Training Step => sample:', sample)
            time.sleep(0.01)

In [161]:
%%timeit
benchmark(FileDataset())

***********************************************************************************
1. __new__ => cls: <class '__main__.FileDataset'>  num_samples: 3
    2. epoch_num: 0
      ------------------------------------------------------------------------------
      3. Opening the File => num_samples: 3
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 0
        ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
        5. Performing Training Step => sample: tf.Tensor([0], shape=(1,), dtype=int64)
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 1
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 2
        ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
        5. Performing Training Step => sample: tf.Tensor([1],

In [162]:
%%timeit
benchmark(FileDataset().prefetch(1))

***********************************************************************************
1. __new__ => cls: <class '__main__.FileDataset'>  num_samples: 3
    2. epoch_num: 0
      ------------------------------------------------------------------------------
      3. Opening the File => num_samples: 3
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 0
        ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
        5. Performing Training Step => sample: tf.Tensor([0], shape=(1,), dtype=int64)
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 1
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 2
        ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
        5. Performing Training Step => sample: tf.Tensor([1],

In [163]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

***********************************************************************************
1. __new__ => cls: <class '__main__.FileDataset'>  num_samples: 3
    2. epoch_num: 0
      ------------------------------------------------------------------------------
      3. Opening the File => num_samples: 3
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 0
        ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
        5. Performing Training Step => sample: tf.Tensor([0], shape=(1,), dtype=int64)
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 1
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 2
        ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
        5. Performing Training Step => sample: tf.Tensor([1],

**As you can notice above, using prefetch improves the performance from 304 ms to 238 and 240 ms**

<h3 style='color:purple'>Cache</h3>

In [178]:
dataset = tf.data.Dataset.range(5)
print('dataset.Dataset.range(5):')
for d in dataset:
    print(d.numpy(), end="; ")
print(); print()
print('dataset.map(lambda x: x**2:')
dataset = dataset.map(lambda x: x**2)
for d in dataset:
    print(d.numpy(), end="; ")
dataset = dataset.cache("mycache.txt")
# The first time reading through the data will generate the data using
# `range` and `map`.
print(); print()
print('dataset.cache("mycache.txt"):')
list(dataset.as_numpy_iterator())

dataset.Dataset.range(5):
0; 1; 2; 3; 4; 

dataset.map(lambda x: x**2:
0; 1; 4; 9; 16; 

dataset.cache("mycache.txt"):


[0, 1, 4, 9, 16]

In [ ]:
# Subsequent iterations read from the cache.
list(dataset.as_numpy_iterator())

In [180]:
# Create dummy map function.
def mapped_function(s):
    # Do some hard pre-processing
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [181]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function), 5)

***********************************************************************************
1. __new__ => cls: <class '__main__.FileDataset'>  num_samples: 3
    2. epoch_num: 0
      ------------------------------------------------------------------------------
      3. Opening the File => num_samples: 3
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 0
        ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
        5. Performing Training Step => sample: tf.Tensor([0], shape=(1,), dtype=int64)
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 1
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 2
        ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
        5. Performing Training Step => sample: tf.Tensor([1],

In [182]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function).cache(), 5)

***********************************************************************************
1. __new__ => cls: <class '__main__.FileDataset'>  num_samples: 3
    2. epoch_num: 0
      ------------------------------------------------------------------------------
      3. Opening the File => num_samples: 3
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 0
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 1
        ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
        5. Performing Training Step => sample: tf.Tensor([0], shape=(1,), dtype=int64)
        rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        4. Reading data => sample_idx: 2
        ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
        5. Performing Training Step => sample: tf.Tensor([1],

**Further reading** https://www.tensorflow.org/guide/data_performance#caching